## Скрейпинг и beautifulsoup4

Лингвисту, особенно компьютерному, часто бывает нужно добывать себе текстовые данные. Интернет - отличный источник таких данных, а программирование позволяет автоматизировать процесс их добычи, так что можно, например, самостоятельно собрать маленький корпус каких-нибудь текстов, взятых из интернета, и проводить на них исследования (карманный ГИКРЯ!). 

Конечно, большие корпуса, созданные по технологии Web as Corpus (Kilgariff), делать сложно, и нужно много всего знать для этого, но небольшой объем материала получить мы с вами вполне можем. 

Отличный источник текстов из интернета - это, конечно, Википедия. Википедия сама выкладывает в открытый доступ дампы для всех своих языков, их можно легко скачать (только они гигантские) [тут](https://dumps.wikimedia.org/). Есть и библиотеки для работы с этими дампами, например, ```pip install wiki-dump-reader```. Вот пример кода для него:

In [ ]:
from wiki_dump_reader import Cleaner, iterate

path = input() # путь к вашему дампу
cleaner = Cleaner()
i = 0
test = open('textwiki.txt', 'w', encoding='utf8') # откроем файл, куда будем записывать обработанный дамп
for title, text in iterate(path): # title - заголовок статьи, text - ее текст
    if i > 10:
        break
    text = cleaner.clean_text(text) # текст нужно почистить от всякой html-обвязки
    cleaned_text, links = cleaner.build_links(text) # можно еще ссылки извлечь из текста
    i += 1
    print(f'text: {cleaned_text}\n___\nLinks: {links}', file=test)
test.close()

Также вы самостоятельно осваивали библиотеку - API к Википедии. (```pip install wikipedia-api```). Комментировать подробно я тут не буду, к тому же, у этой библиотеки хорошая документация, просто приведу некоторые примеры из нее. 

In [ ]:
import wikipediaapi

In [ ]:
# Получаем одиночную страницу
wiki_wiki = wikipediaapi.Wikipedia('en')
page_py = wiki_wiki.page('Python_(programming_language)')

In [ ]:
# проверяем, существует ли вообще такая:

page_py = wiki_wiki.page('Python_(programming_language)')
print(f"Page - Exists: {page_py.exists()}")
# Page - Exists: True

page_missing = wiki_wiki.page('NonExistingPageWithStrangeName')
print(f"Page - Exists: {page_missing.exists()}")
# Page - Exists: False

In [ ]:
# выводим информацию о странице:
print(page_py.title)
print(page_py.summary[0:60])

In [ ]:
# как получать полный текст страницы:

wiki_wiki = wikipediaapi.Wikipedia(
        language='en',
        extract_format=wikipediaapi.ExtractFormat.WIKI
)

p_wiki = wiki_wiki.page("Test 1")
print(p_wiki.text)


wiki_html = wikipediaapi.Wikipedia(
        language='en',
        extract_format=wikipediaapi.ExtractFormat.HTML
)
p_html = wiki_html.page("Test 1")
print(p_html.text)

In [ ]:
# как получить отдельные секции страницы:

def print_sections(sections, level=0):
    for s in sections:
        print(f"{level + 1}: {s.title} - {s.text[0:40]}")
        print_sections(s.sections, level + 1)

print_sections(page_py.sections)

In [ ]:
# как получить ссылки на эту же страницу на других языках:

def print_langlinks(page):
    langlinks = page.langlinks
    for k in sorted(langlinks.keys()):
        v = langlinks[k]
        print(f"{k}: {v.language} - {v.title}: {v.fullurl}")

print_langlinks(page_py)

In [ ]:
# как получить ссылки на странице:

def print_links(page):
    links = page.links
    for title in sorted(links.keys()):
        print(f"{title}: {links[title]}")

print_links(page_py)

In [ ]:
# Как получить категории страницы:

def print_categories(page):
    categories = page.categories
    for title in sorted(categories.keys()):
        print(f"{title}: {categories[title]}")


print("Categories")
print_categories(page_py)

In [ ]:
# как получить все страницы в категории:

def print_categorymembers(categorymembers, level=0, max_level=1):
    for c in categorymembers.values():
        print(f"{level + 1}: {c.title} (ns: c.ns)")
        if c.ns == wikipediaapi.Namespace.CATEGORY and level < max_level:
            print_categorymembers(c.categorymembers, level=level + 1, max_level=max_level)


cat = wiki_wiki.page("Category:Physics")
print("Category members: Category:Physics")
print_categorymembers(cat.categorymembers)

Наконец, в питоне есть встроенная библиотека urllib, которая позволяет обращаться к любым страницам в интернете и добывать их содержимое (даже точнее, ее подмодуль urllib.request). Самый простой способ с ее помощью прочитать содержимое страницы выглядит так:

In [ ]:
import urllib.request
with urllib.request.urlopen('http://python.org/') as response:
    html = response.read()

Выглядеть, правда, считанное будет страшновато:

In [29]:
html[:500]

b'<!doctype html>\n<!--[if lt IE 7]>   <html class="no-js ie6 lt-ie7 lt-ie8 lt-ie9">   <![endif]-->\n<!--[if IE 7]>      <html class="no-js ie7 lt-ie8 lt-ie9">          <![endif]-->\n<!--[if IE 8]>      <html class="no-js ie8 lt-ie9">                 <![endif]-->\n<!--[if gt IE 8]><!--><html class="no-js" lang="en" dir="ltr">  <!--<![endif]-->\n\n<head>\n    <!-- Google tag (gtag.js) -->\n    <script async src="https://www.googletagmanager.com/gtag/js?id=G-TF35YF9CVH"></script>\n    <script>\n      window.d'

Собственно говоря, тут неплохо вспомнить, что в интернете почти все страницы на самом деле написаны в каком-нибудь html. Язык разметки html (иногда его называют языком программирования, но это спорный вопрос, многие программисты будут с пеной у рта доказывать, что НЕТ) довольно просто устроен, в основном у него есть теги, которые берутся в треугольные скобки. В большинстве теги должны открываться и закрываться. Один из самых простых тегов, например, тег для курсива: ```<i> что-то, написанное курсивом </i>```. Другой тег (одиночный) - для переноса на новую строку: ```</br>```. (Кстати, тетрадки юпитера умеют в html...)

И вот где-то там, посреди html-ного мусора, зарыты наши с вами тексты, которые мы и хотим добыть...

In [28]:
html[3000:4000]

b'h-icon-72x72-precomposed.png">\n    <link rel="apple-touch-icon-precomposed" href="/static/apple-touch-icon-precomposed.png">\n    <link rel="apple-touch-icon" href="/static/apple-touch-icon-precomposed.png">\n\n    \n    <meta name="msapplication-TileImage" content="/static/metro-icon-144x144-precomposed.png"><!-- white shape -->\n    <meta name="msapplication-TileColor" content="#3673a5"><!-- python blue -->\n    <meta name="msapplication-navbutton-color" content="#3673a5">\n\n    <title>Welcome to Python.org</title>\n\n    <meta name="description" content="The official home of the Python Programming Language">\n    <meta name="keywords" content="Python programming language object oriented web free open source software license documentation download community">\n\n    \n    <meta property="og:type" content="website">\n    <meta property="og:site_name" content="Python.org">\n    <meta property="og:title" content="Welcome to Python.org">\n    <meta property="og:description" content="

Как же их достать оттуда? Можно, конечно, регулярными выражениями: но это долго и муторно, и для каждой страницы регулярные выражения могут понадобиться свои. Ну, и поскольку эта задача частая и хорошо известная, разумеется, для нее написаны специальные библиотеки. 

Самая известная такая библиотека - beautifulsoup4. Вообще у этой библиотеки тоже очень хорошая подробная [документация](https://www.crummy.com/software/BeautifulSoup/), но какие-то основные штуки мы с вами сейчас посмотрим. 

Устанавливается как 

    pip install beautifulsoup4
    
Вероятно, в жизни вам еще понадобятся дополнительные библиотеки:

    pip install lxml
    pip install html5lib
    
(если вдруг вам надо будет извлекать текст из этих форматов).

Основной класс библиотеки bs4 - это так называемый суп (BeautifulSoup). Когда мы хотим распарсить какую-то html-страницу, нужно ее содержимое в него передать. Давайте передадим нашу страницу с сайта питона:

In [ ]:
from bs4 import BeautifulSoup as bs

soup = bs(html, 'html.parser')

Вот теперь с нашим супом можно сделать все, что угодно. Например, можно извлечь заголовок страницы:

In [25]:
soup.title

<title>Welcome to Python.org</title>

Как правило, почти у любого объекта внутри супа есть атрибут text, который позволяет добыть сам текст без html-тегов:

In [26]:
soup.title.text

'Welcome to Python.org'

Частая задача - найти все ссылки на нашей странице. Это можно сделать таким образом:

In [ ]:
for link in soup.find_all('a'):
    print(link.get('href'))

Там могут быть как полные ссылки на внешние страницы, так и относительные: обычно если ссылка начинается на /, это значит, что к ней спереди нужно добавить исходный адрес страницы. 

Вообще прежде чем что-нибудь еще вытаскивать из документа, давайте прикинем, какая у нашего html структура. 

<img src="https://itchief.ru/assets/images/javascript/dom/tree.png" width=700 />

Примерно так (кстати, картинка вставлена html-кодом :)). На самом деле внутри body могут быть какие угодно штуки, могут быть ```<div>``` (типа сегменты), например. К любому из этих тегов можно обратиться через soup. Например, выше, когда мы написали ```soup.title```, мы ровно это и сделали! 

Окей, заголовок документа и его тело (body) явно присутствуют в одиночном экземпляре, но как быть с разнообразными ```<div>```, которых может быть больше одного? Если мы напишем просто ```soup.div```, мы обратимся к первому встретившемуся такому тегу. 

In [ ]:
soup.div

(Там был очень длинный div, поэтому я почистила аутпут ячейки). 

Давайте возьмем что-нибудь покороче:

In [32]:
soup.a

<a href="#content" title="Skip to content">Skip to content</a>

Это самая первая ссылка на странице. На самом деле ```soup.a``` - это специальный объект класса Tag, у которого тоже есть всякие свои штуки, которые мы можем посмотреть. Например, у тега есть имя:

In [33]:
soup.a.name

'a'

(Довольно очевидное...)

Еще у него могут быть атрибуты, к которым можно обращаться, как к ключам словаря. У нашей ссылки есть title:

In [35]:
soup.a['title']

'Skip to content'

Атрибуты можно добавлять, удалять и изменять (но для задач добывания текста из html-страницы нам это не особо нужно). 

Добыть все теги с одинаковым именем в нашем супе мы уже пробовали, для этого есть метод find_all. При этом может понадобиться проверять, какие атрибуты есть у наших тегов. Для этого есть метод has_attr.

In [49]:
for div in soup.find_all('div'):
    if div.has_attr('class'):
        print(div['class'])

['do-not-print']
['top-bar', 'do-not-print']
['skip-link', 'screen-reader-text']
['container']
['options-bar-container', 'do-not-print']
['options-bar']
['adjust-font-size']
['winkwink-nudgenudge']
['header-banner']
['flex-slideshow', 'slideshow']
['slide-code']
['slide-copy']
['slide-code']
['slide-copy']
['slide-code']
['slide-copy']
['slide-code']
['slide-copy']
['slide-code']
['slide-copy']
['introduction']
['content-wrapper']
['container']
['row']
['small-widget', 'get-started-widget']
['small-widget', 'download-widget']
['small-widget', 'documentation-widget']
['small-widget', 'jobs-widget', 'last']
['list-widgets', 'row']
['medium-widget', 'blog-widget']
['shrubbery']
['medium-widget', 'event-widget', 'last']
['shrubbery']
['row']
['medium-widget', 'success-stories-widget']
['shrubbery']
['success-story-item']
['medium-widget', 'applications-widget', 'last']
['shrubbery']
['pep-widget']
['psf-widget']
['python-logo']
['main-footer-links']
['container']
['site-base']
['container'

Ну и самая важная для нас штука, которая позволит извлечь из тега человекочитаемый текст: get_text(). 

In [53]:
for idx, div in enumerate(soup.find_all('div')): # пронумерую просто для того, чтобы не выводить вообще все
    if idx >= 5:
        break
    print(div.get_text()[:100].replace('\n', ' ')) # навешаю всякого, чтобы не выводить полный текст и убрать лишние переносы строк

  Notice: While JavaScript is not essential for this website, your interaction with the content will
 Notice: While JavaScript is not essential for this website, your interaction with the content will 
   Skip to content   ▼ Close                    Python   PSF   Docs   PyPI   Jobs   Community    ▲ T
 Skip to content 
     Donate  ≡ Menu   Search This Site                                       GO                     


Что на самом деле делает этот метод - он берет все кусочки человеческого текста в коде и склеивает их, как ему кажется правильным. Мы можем достать эти кусочки самостоятельно:

In [54]:
' '.join([text for text in soup.stripped_strings]) # stripped_strings достанет все человекочитаемые кусочки текста

'Welcome to Python.org Notice: While JavaScript is not essential for this website, your interaction with the content will be limited. Please turn JavaScript on for the full experience. Skip to content ▼ Close Python PSF Docs PyPI Jobs Community ▲ The Python Network Donate ≡ Menu Search This Site GO A A Smaller Larger Reset Socialize Facebook Twitter Chat on IRC About Applications Quotes Getting Started Help Python Brochure Downloads All releases Source code Windows macOS Other Platforms License Alternative Implementations Documentation Docs Audio/Visual Talks Beginner\'s Guide Developer\'s Guide FAQ Non-English Docs PEP Index Python Books Python Essays Community Diversity Mailing Lists IRC Forums PSF Annual Impact Report Python Conferences Special Interest Groups Python Logo Python Wiki Code of Conduct Community Awards Get Involved Shared Stories Success Stories Arts Business Education Engineering Government Scientific Software Development News Python News PSF Newsletter PSF News PyCon

Это, разумеется, не все возможности библиотеки, но для начала работы с ней хватит. Все подробности можно найти в их документации. 